In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [1]:
import numpy as np

In [2]:
import sys
sys.path.append('..')

Import Enzyme File

In [3]:
import helper
X,y,enz_names = helper.parseEnzymeFile('../Data/Enzyme_aligned.txt')

Import Best Positions

In [4]:
bestposfile = '../AutoPos/BestPos3.txt'
bestpos = []
with open(bestposfile,'r') as f:
    for lines in f:
        bestpos.append(int(lines.strip()))

Get only the best positions among all others

In [5]:
X_pos = np.array(list(map(list,X)))

In [6]:
X_sel_pos = X_pos[:,bestpos]

train-test-split 

In [7]:
from sklearn.model_selection import train_test_split,GridSearchCV

In [8]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X_sel_pos,y,test_size=0.25,random_state=7)

Try OHE

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(Xtrain)

OneHotEncoder(handle_unknown='ignore')

In [11]:
Xtr,Xte = ohe.transform(Xtrain).toarray(),ohe.transform(Xtest).toarray()

pipeline

In [12]:
from mySVM.model import SVM

In [13]:
svm = SVM(Xtr,Xte,ytrain,ytest,optimize=True,verbose=True)

-----Initial Model Evaluation-----
-----Training Accuracy:0.7848101265822784-----
-----Testing Accuracy:0.4074074074074074-----
-----Hyperparameter Optimization-----
Fitting 3 folds for each of 72 candidates, totalling 216 fits


/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1974s.) Setting batch_size=2.


score = 0.41
{'SVM__C': 20, 'SVM__gamma': 0.1, 'SVM__kernel': 'rbf', 'pca__n_components': 20}


[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1297s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:   13.3s finished
/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Try other encodings

get enzymes sequence from 155 to 325, apply kmer motif builder

In [14]:
X_sel = X_pos[:,155:325]

In [15]:
X_sel_str = np.array([''.join(x) for x in X_sel])#.reshape(-1,1)

In [16]:
from FeatEngg.ngramEnc import Ngram

In [17]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X_sel_str,y,test_size=0.25,random_state=7)

In [18]:
ngenc = Ngram(Xtrain,7,1)

In [19]:
ngenc.fit()

In [20]:
Xtr,Xte = ngenc.transform(Xtrain),ngenc.transform(Xtest)

In [21]:
svm = SVM(Xtr,Xte,ytrain,ytest,optimize=True,verbose=True)

/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0251s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0251s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.1s


-----Initial Model Evaluation-----
-----Training Accuracy:0.8607594936708861-----
-----Testing Accuracy:0.6296296296296297-----
-----Hyperparameter Optimization-----
Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1391s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 191 out of 216 | elapsed:    0.4s remaining:    0.1s


score = 0.67
{'SVM__C': 1, 'SVM__gamma': 'scale', 'SVM__kernel': 'rbf', 'pca__n_components': 40}


[Parallel(n_jobs=-1)]: Done 213 out of 216 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:    0.4s finished
/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [22]:
from FeatEngg.gaacEnc import GAAC

In [23]:
X_gaac = GAAC().transform(X_sel_str)

In [24]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X_gaac,y,test_size=0.25,random_state=7)

In [25]:
nggaac = Ngram(Xtrain,7,1)

In [26]:
nggaac.fit()

In [27]:
Xtr,Xte = nggaac.transform(Xtrain),nggaac.transform(Xtest)

In [28]:
svm = SVM(Xtr,Xte,ytrain,ytest,pca_comp=40,regC=20,kern='linear',verbose=True)

-----Initial Model Evaluation-----
-----Training Accuracy:0.7721518987341772-----
-----Testing Accuracy:0.7407407407407407-----


/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [29]:
svm.ypredtest

array([3, 3, 3, 1, 2, 2, 2, 2, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 3,
       3, 1, 1, 1, 3])

In [30]:
ytest

array([3, 3, 3, 1, 2, 3, 3, 2, 3, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 3, 3,
       2, 1, 1, 1, 3])

In [31]:
svm.acc_test,svm.acc_train

(0.7407407407407407, 0.7721518987341772)

In [32]:
from helper import modified_split
from IndModels.models import GAACModel

In [33]:
def multiprocess_func(randomseed):
    np.random.seed(randomseed)
    test_idx_i,train_idx_i = modified_split(enz_names,'Data/Enzyme-with-accession.txt')
    
    gaacModel_i = GAACModel(enz_names,X,y,train_idx_i,test_idx_i,pca_components=40,regCparam=20,kernparam='linear')
    
    
    return gaacModel_i.SVMobject.acc_test,gaacModel_i.SVMobject.acc_train


In [34]:
import multiprocessing as mp
print(mp.cpu_count())
pool = mp.Pool(mp.cpu_count())

24


/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/svm/_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=30).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [35]:
%%time
accs = list(pool.map(multiprocess_func,[i for i in range(10000)]))

In [36]:
te_accs = [acc[0] for acc in accs]

In [37]:
min(te_accs),np.mean(te_accs),np.std(te_accs)

(0.36, 0.734936, 0.08809969298470909)